In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 96 × Intel(R) Xeon(R) Gold 5318Y CPU @ 2.10GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, icelake-server)
Threads: 1 default, 0 interactive, 1 GC (on 96 virtual cores)
Environment:
  JULIA_PKG_SERVER = https://mirrors.tuna.tsinghua.edu.cn/julia


In [1]:
@time using CSV, DataFrames, Distributions, Plots, ProgressMeter
@time using Random, Statistics, StatsPlots, Turing

  5.586627 seconds (2.15 M allocations: 130.029 MiB, 7.60% gc time, 3.58% compilation time: 78% of which was recompilation)
  4.973507 seconds (1.54 M allocations: 94.065 MiB, 2.47% gc time, 7.11% compilation time: 65% of which was recompilation)


# 1 Load a tsv file

In [2]:
folder_path = "/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/02_allele_file"

column_types = Dict("chrom" => String, "pos_raw" => Int32, 
        "ref" => String, "context" => String, "alt" => String, 
        "support_reads" => Int32, 
        "depth" => Int32, "freq" => Float32)
#Char, String3, String7 are not supported by Parquet.
sample_id="sample20240318-C-T7-R-20240315-C61_I1"
file_path = joinpath(folder_path, "$(sample_id).allele.txt");
@time df_raw = CSV.read(file_path, DataFrame, types=column_types)

first(df_raw,3)

  7.598030 seconds (14.24 M allocations: 700.154 MiB, 14.56% gc time, 49.48% compilation time: 33% of which was recompilation)


Row,chrom,pos_raw,ref,context,alt,support_reads,depth,freq
,String,Int32,String,String,String,Int32,Int32,Float32
1,chr1,11168282,T,GTT,T,365,365,1.0
2,chr1,11168282,T,GTT,C,0,365,0.0
3,chr1,11168282,T,GTT,G,0,365,0.0


In [3]:
Base.summarysize(df_raw)

93221325

# 2 JLD (not working for hdf5)

In [3]:
@time using JLD

  0.624635 seconds (480.16 k allocations: 32.139 MiB, 52.54% compilation time: 71% of which was recompilation)


In [13]:
# JLD does not work.
#hdf5_path = joinpath(folder_path, "$(sample_id).allele.h5");
#JLD.save(hdf5_path, df_raw)

Error encountered while save FileIO.File{FileIO.DataFormat{:HDF5}, String}("/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.h5").

Fatal error:


LoadError: MethodError: no method matching fileio_save(::FileIO.File{FileIO.DataFormat{:HDF5}, String}, ::DataFrame)

[0mClosest candidates are:
[0m  fileio_save(::FileIO.File{FileIO.DataFormat{:HDF5}}, [91m::AbstractDict[39m; track_order, kwargs...)
[0m[90m   @[39m [33mHDF5[39m [90m~/.julia/packages/HDF5/Z859u/src/[39m[90m[4mfileio.jl:58[24m[39m

Stacktrace:
 [1] [0m[1m#invokelatest#2[22m
[90m   @[39m [90m./[39m[90m[4messentials.jl:892[24m[39m[90m [inlined][39m
 [2] [0m[1minvokelatest[22m
[90m   @[39m [90m./[39m[90m[4messentials.jl:889[24m[39m[90m [inlined][39m
 [3] [0m[1maction[22m[0m[1m([22m[90mcall[39m::[0mSymbol, [90mlibraries[39m::[0mVector[90m{Union{Base.PkgId, Module}}[39m, [90mfile[39m::[0mFileIO.Formatted, [90margs[39m::[0mDataFrame; [90moptions[39m::[0m@Kwargs[90m{}[39m[0m[1m)[22m
[90m   @[39m [32mFileIO[39m [90m~/.julia/packages/FileIO/xOKyx/src/[39m[90m[4mloadsave.jl:219[24m[39m
 [4] [0m[1mact

In [12]:
?JLD.save

  * `save(filename, data...)` saves the contents of a formatted file, trying to infer the format from `filename`.
  * `save(Stream{format"PNG"}(io), data...)` specifies the format directly, and bypasses the format [`query`](@ref).
  * `save(File{format"PNG"}(filename), data...)` specifies the format directly, and bypasses the format [`query`](@ref).
  * `save(f, data...; options...)` passes keyword arguments on to the saver.


# 3 Parquet.jl

In [2]:
@time using Parquet

  0.349785 seconds (153.01 k allocations: 7.514 MiB)


## 3.1 Write by Parquet.jl

In [9]:
parquet_path = joinpath(folder_path, "$(sample_id).allele.parquet");
@time Parquet.write_parquet(parquet_path, df_raw)

  4.284007 seconds (13.23 M allocations: 672.631 MiB, 10.55% gc time, 67.19% compilation time)


4

In [10]:
?Parquet.write_parquet

```
Write a parquet file from a Tables.jl compatible table e.g DataFrame
```

io                  -   A writable IO stream tbl                 -   A Tables.jl columnaccessible table e.g. a DataFrame compression_code    -   Default "SNAPPY". The compression codec. The supported                         values are "UNCOMPRESSED", "SNAPPY", "ZSTD", "GZIP"

---

```
Write a parquet file from a Tables.jl compatible table e.g DataFrame
```

path                -   The file path tbl                 -   A Tables.jl columnaccessible table e.g. a DataFrame compression_code    -   Default "SNAPPY". The compression codec. The supported                         values are "UNCOMPRESSED", "SNAPPY", "ZSTD", "GZIP"


In [46]:
@time Parquet.write_parquet(joinpath(folder_path, "$(sample_id).allele.zstd.parquet"), 
    df_raw, compression_code="zstd")

LoadError: MethodError: no method matching write_parquet(::String, ::DataFrame; compression_code::String)

[0mClosest candidates are:
[0m  write_parquet(::Any, ::Any; compression_codec)[91m got unsupported keyword argument "compression_code"[39m
[0m[90m   @[39m [35mParquet[39m [90m~/.julia/packages/Parquet/6tj1X/src/[39m[90m[4mwriter.jl:524[24m[39m
[0m  write_parquet([91m::IO[39m, ::Any; compression_codec)[91m got unsupported keyword argument "compression_code"[39m
[0m[90m   @[39m [35mParquet[39m [90m~/.julia/packages/Parquet/6tj1X/src/[39m[90m[4mwriter.jl:460[24m[39m


## 3.2 Read a parquet file

In [56]:
@time pq_ds = Parquet.read_parquet(parquet_path)
@show typeof(pq_ds)
@time @show Tables.schema(pq_ds)  # view table schema
@time df_parquet = DataFrame(pq_ds)
first(df_parquet, 3)

  0.005970 seconds (1.10 k allocations: 44.578 KiB)
typeof(pq_ds) = Parquet.Table
Tables.schema(pq_ds) = Tables.Schema:
 :chrom          Union{Missing, String}
 :pos_raw        Union{Missing, Int32}
 :ref            Union{Missing, String}
 :context        Union{Missing, String}
 :alt            Union{Missing, String}
 :support_reads  Union{Missing, Int32}
 :depth          Union{Missing, Int32}
 :freq           Union{Missing, Float32}
  0.016040 seconds (2.81 k allocations: 1.598 MiB)
 27.583395 seconds (23.31 M allocations: 1.426 GiB, 95.38% gc time, 0.06% compilation time)


Row,chrom,pos_raw,ref,context,alt,support_reads,depth,freq
,String?,Int32?,String?,String?,String?,Int32?,Int32?,Float32?
1,chr1,11168282,T,GTT,T,365,365,1.0
2,chr1,11168282,T,GTT,C,0,365,0.0
3,chr1,11168282,T,GTT,G,0,365,0.0


In [53]:
@time Base.summarysize(df_parquet)

 49.651417 seconds (46.60 M allocations: 1.539 GiB, 7.72% gc time)


272344192

# 4 Parquet2.jl

## 4.1 Write a Parquet2 file

In [14]:
using Parquet2, Tables, DataFrames

In [69]:
parquet2_path = joinpath(folder_path, "$(sample_id).allele.parquet2");
@time Parquet2.writefile(parquet2_path, df_raw)

LoadError: ArgumentError: type Char does not have a corresponding parquet type

In [41]:
?Parquet2.writefile

```
writefile(io::IO, path, tbl; kw...)
writefile(path, tbl; kw...)
```

Write the Tables.jl compatible table `tbl` to the `IO` or the file at `path`.  Note that the path is used in parquet metadata, which is why it is possible to specify the path separately from the `io` stream.  See [`FileWriter`](@ref) for a description of all possible arguments.

This function writes a file all in one call.  Files will be written as one parquet row group per table partition. An intermediate [`FileWriter`](@ref) object is used.


In [49]:
@time Parquet2.writefile(joinpath(folder_path, "$(sample_id).allele.zstd.parquet2"), 
    df_raw, compress=:zstd)

LoadError: ArgumentError: "compress" is an invalid keyword argument for Parquet2.WriteOptions

## 4.2 Parquet2 reads a file created by Parquet.jl

In [57]:
@time ds = Parquet2.Dataset(parquet_path)
@time @show Tables.schema(ds)  # view table schema
@time t = Tables.columntable(ds)  # load as a NamedTuple of columns
@time df = DataFrame(ds; copycols=false)  # load entire dataset as a DataFrame
first(df, 3)
#df1 = DataFrame(ds[1]; copycols=false)  # load first RowGroup as a DataFrame

  0.000798 seconds (1.45 k allocations: 110.469 KiB)
Tables.schema(ds) = Tables.Schema:
 :chrom          Union{Missing, String}
 :pos_raw        Union{Missing, Int32}
 :ref            Union{Missing, String}
 :context        Union{Missing, String}
 :alt            Union{Missing, String}
 :support_reads  Union{Missing, Int32}
 :depth          Union{Missing, Int32}
 :freq           Union{Missing, Float32}
  0.015601 seconds (2.96 k allocations: 1.609 MiB)
 11.434261 seconds (11.65 M allocations: 663.215 MiB, 92.63% gc time)
  5.111808 seconds (11.65 M allocations: 663.222 MiB, 82.72% gc time)


Row,chrom,pos_raw,ref,context,alt,support_reads,depth,freq
,String?,Int32?,String?,String?,String?,Int32?,Int32?,Float32?
1,chr1,11168282,T,GTT,T,365,365,1.0
2,chr1,11168282,T,GTT,C,0,365,0.0
3,chr1,11168282,T,GTT,G,0,365,0.0


In [30]:
#not sure why this operation takes so long for data structures
#  created by Parquet2.jl from files generated by Parquet.jl.
@time @show Base.summarysize(t)
@time @show Base.summarysize(df)

Base.summarysize(t) = 272343528
 46.959012 seconds (46.60 M allocations: 1.539 GiB, 5.48% gc time)
Base.summarysize(df) = 272344192
 46.709742 seconds (46.60 M allocations: 1.539 GiB, 8.90% gc time)


272344192

In [22]:
@show typeof(t)

sch = Tables.Schema:
 :chrom          Union{Missing, String}
 :pos_raw        Union{Missing, Int32}
 :ref            Union{Missing, String}
 :context        Union{Missing, String}
 :alt            Union{Missing, String}
 :support_reads  Union{Missing, Int32}
 :depth          Union{Missing, Int32}
 :freq           Union{Missing, Float32}
typeof(t) = @NamedTuple{chrom::Vector{Union{Missing, String}}, pos_raw::Vector{Union{Missing, Int32}}, ref::Vector{Union{Missing, String}}, context::Vector{Union{Missing, String}}, alt::Vector{Union{Missing, String}}, support_reads::Vector{Union{Missing, Int32}}, depth::Vector{Union{Missing, Int32}}, freq::Vector{Union{Missing, Float32}}}


@NamedTuple{chrom::Vector{Union{Missing, String}}, pos_raw::Vector{Union{Missing, Int32}}, ref::Vector{Union{Missing, String}}, context::Vector{Union{Missing, String}}, alt::Vector{Union{Missing, String}}, support_reads::Vector{Union{Missing, Int32}}, depth::Vector{Union{Missing, Int32}}, freq::Vector{Union{Missing, Float32}}}

## 4.3 Parquet2 reads a file created by itself

In [58]:
@time ds2 = Parquet2.Dataset(parquet2_path)
@show typeof(ds2)
@time @show Tables.schema(ds2)  # view table schema
@time t2 = Tables.columntable(ds2)  # load as a NamedTuple of columns
@time df2 = DataFrame(ds2; copycols=false)  # load entire dataset as a DataFrame
first(df2,3)
#df1 = DataFrame(ds[1]; copycols=false)  # load first RowGroup as a DataFrame

  0.004855 seconds (1.48 k allocations: 111.875 KiB)
typeof(ds2) = Parquet2.Dataset{Parquet2.FileManager{FilePathsBase.PosixPath}}
Tables.schema(ds2) = Tables.Schema:
 :chrom          String
 :pos_raw        Int32
 :ref            String
 :context        String
 :alt            String
 :support_reads  Int32
 :depth          Int32
 :freq           Float32
  0.000325 seconds (696 allocations: 38.570 KiB)
  7.241007 seconds (841 allocations: 201.789 MiB, 92.07% gc time)
  6.982388 seconds (967 allocations: 201.797 MiB, 91.90% gc time)


Row,chrom,pos_raw,ref,context,alt,support_reads,depth,freq
,String,Int32,String,String,String,Int32,Int32,Float32
1,chr1,11168282,T,GTT,T,365,365,1.0
2,chr1,11168282,T,GTT,C,0,365,0.0
3,chr1,11168282,T,GTT,G,0,365,0.0


In [59]:
@time @show Base.summarysize(t2)
@time @show Base.summarysize(df2)

Base.summarysize(t2) = 93215285
  0.000172 seconds (404 allocations: 18.578 KiB)
Base.summarysize(df2) = 93215917
  0.000151 seconds (422 allocations: 19.391 KiB)


93215917

## 4.4 Convert all PanelM's txt files to parquet

In [64]:
# DocStringExtensions provides $(SIGNATURES)
using DocStringExtensions
"""
Convert all ".txt" files in the folder into .parquet via Parquet2.jl

$(SIGNATURES)


## Return values:
- no of converted files.

Type String7, String3 from package InlineString is the smallest InlineString type.
"""
function convert_tsv_files_to_parquet(input_folder_path::String, 
        output_folder_path::String, sequencer_type::String="T7")
    filename_v = readdir(input_folder_path);
    no_of_files = length(filename_v);
    println("Starting to convert $(no_of_files) tsv files from $(input_folder_path) to parquet ")

    column_types = Dict("chrom" => String, "pos_raw" => Int32, 
            "ref" => String, "context" => String, "alt" => String, 
            "support_reads" => Int32, 
            "depth" => Int32, "freq" => Float32)
    #Char, String3, String7 are not supported by Parquet.
    no_of_converts = 0 
    @showprogress for i in eachindex(filename_v);
        filename = filename_v[i];
        file_path = joinpath(input_folder_path, filename);
        #println(i, '/', no_of_files);
        to_read = false;
        endswith(filename, ".allele.txt") && (to_read = true)
        sample_id = split(filename, ".")[1]
        if to_read == true
            df_raw = CSV.read(file_path, DataFrame, types=column_types)
            parquet2_path = joinpath(output_folder_path, "$(sample_id).allele.parquet");
            Parquet2.writefile(parquet2_path, df_raw)
            no_of_converts += 1
        end
    end
    println("Done converting $(no_of_converts) tsv files to parquet in $(output_folder_path).");
    return no_of_converts
end

convert_tsv_files_to_parquet

In [65]:
parent_folder = "/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/"
input_folder_path = joinpath(parent_folder, "02_allele_file")
output_folder_path = joinpath(parent_folder, "02_allele_file_parquet")

@time convert_tsv_files_to_parquet(input_folder_path, output_folder_path)

Ready to read files from /mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/02_allele_file


Progress: 100%|█████████████████████████████████████████| Time: 0:44:36


139

# 5 Arrow
- https://arrow.apache.org/julia/stable/manual

In [32]:
using Arrow

In [37]:
arrow_path = joinpath(folder_path, "$(sample_id).allele.arrow");
@time Arrow.write(arrow_path, df2)

  2.147883 seconds (34.95 M allocations: 722.262 MiB, 13.87% gc time)


"/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.arrow"

In [40]:
arrow_lz4_path = joinpath(folder_path, "$(sample_id).allele.lz4.arrow");
@time Arrow.write(arrow_lz4_path, df2; compress=:lz4)

  2.673021 seconds (34.95 M allocations: 778.024 MiB)


"/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.lz4.arrow"

In [38]:
arrow_zstd_path = joinpath(folder_path, "$(sample_id).allele.zstd.arrow");
@time Arrow.write(arrow_zstd_path, df2; compress=:zstd)

  2.198118 seconds (34.95 M allocations: 777.829 MiB, 11.16% gc time)


"/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets/02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.zstd.arrow"

In [61]:
arrow_parquet_fpath = Parquet2.filepath(joinpath(output_folder_path, 
        "$(sample_id).allele.arrow.parquet"))
@time Arrow.write(arrow_parquet_fpath, df2; compress=:zstd)

LoadError: MethodError: no method matching filepath(::String)

[0mClosest candidates are:
[0m  filepath([91m::Parquet2.PageIterator[39m)
[0m[90m   @[39m [36mParquet2[39m [90m~/.julia/packages/Parquet2/ZssLg/src/[39m[90m[4mschema.jl:1017[24m[39m
[0m  filepath([91m::Parquet2.Column[39m)
[0m[90m   @[39m [36mParquet2[39m [90m~/.julia/packages/Parquet2/ZssLg/src/[39m[90m[4mschema.jl:812[24m[39m


# 6 File size comparison

- File sizes:
- .arrow with :zstd is smallest. 20X smaller than the raw txt.

- huang.yu@Cc41Saturn:/mnt/titan01/IVD/projects/LDT/jinxiang/03_zhong_panel/05_build_like_MRD_BG/01_build_datasets$ ls -shl 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.* 
- 29M -rw-rw-r--+ 1 huang.yu      basic_bioinfo  56M May  7 17:31 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.arrow
- 8.9M -rw-rw-r--+ 1 huang.yu      basic_bioinfo 9.7M May  7 17:31 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.lz4.arrow
- 15M -rw-rw-r--+ 1 huang.yu      basic_bioinfo  22M May  7 15:07 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.parquet
- 11M -rw-rw-r--+ 1 huang.yu      basic_bioinfo  14M May  7 15:54 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.parquet2
- 33M -rw-rw-r--+ 1 chen.jinxiang trans_bioinfo 107M May  6 16:50 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.txt
- 4.9M -rw-rw-r--+ 1 huang.yu      basic_bioinfo 5.4M May  7 17:31 02_allele_file/sample20240318-C-T7-R-20240315-C61_I1.allele.zstd.arrow



- .parquet or .parquet2 files can be viewed by datafrusion-cli, https://datafusion.apache.org/user-guide/cli/datasources.html or https://lib.rs/crates/pqrs.
- csv2arrow, csv2parquet, json2arrow, json2parquet, https://github.com/domoritz/arrow-tools